In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
def get_price(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  price = soup.find('span',{'itemprop': 'price'})['content']
  return price


In [5]:
print(get_price('https://www.ebay.com/itm/393514670181?_trkparms=amclksrc%3DITM%26aid%3D1110010%26algo%3DHOMESPLICE.DISCCARDS%26ao%3D1%26asc%3D20200818141627%26meid%3D35d3db7929d24681ae460bcffcef9ec3%26pid%3D101110%26rk%3D1%26rkt%3D1%26sd%3D234145481103%26itm%3D393514670181%26pmt%3D1%26noa%3D0%26pg%3D2563228%26algv%3Ddefault%26brand%3DPUMA&_trksid=p2563228.c101110.m1982&amdata=cksum%3A39351467018135d3db7929d24681ae460bcffcef9ec3%7Cenc%3AAQAGAAAB4LAfpiaX1XNFiDFtpTEdZWAf3k%252BG0cxQdDyso%252BsfnJRewPxG54S3DsmowvuLtaYAL074LDQn8f4BO%252FQ4tnhMeCvyvItOMvn9aGVeR0jIgBiCJKa5jp6Qg1tJFMDfSONIILjnzUX%252FKOKhppxys0f9I%252BhBJ7txJuN4sF4CAUDbMdW7mKbQbYhpUMJi5Ezm%252FRh7WDuJ%252FuD%252FWp4JgcihztnZkJJdE3CEvi5CylVvWmrSLJX30dPOgNFdI%252BBfMJY4vegu%252FqE%252FUIaUVAg8GtZxGtDvMSj59qitBkZt%252F2Ca%252BOr8%252FHCTDBnDMklT7uinO3rowg3kguth%252FHYBMbLNHLR4zlcxS01QbVicZjnG7WlsxuvEcwwVuO0r3zPQ0oYhrjy0P1PuKktzVwA2ZySbCaTEolkwcuJZhNRyJNwgoTAAcn3c8t9FFMoPrjKC%252B9sEi5xsQbxCP33HknaJY1JzR05ZbcHyRWDyfj7YQViK9%252BN%252F4wAQDqkzxpncA6zwjCBmRefCVqs5%252BseobqNDkB3NM358%252BasmsJShdxMnLtDFo1%252BIRRPztK2GZTDxSlBS7PjPZAS2lZH0CJE4mBgBqa2VhF1hY%252Fm%252FEjh8uwEqY%252FTZD484aCkIXKN%252F7ztdjmv4vPtI1n4MPc42sw%253D%253D%7Campid%3APL_CLK%7Cclp%3A2563228'))

82.99


### Using selenium


In [2]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('useAutomationExtention', False)
chrome_options.add_argument('--ifnore-certificate-errors')
chrome_options.add_argument('--start-maximized')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [6]:
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd 
import os
import time

In [4]:
#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
#FireFoxDriverPath = os.path.join(os.getcwd(), 'Drivers', 'geckodriver.exe')
#FireFoxProfile = webdriver.FirefoxProfile()
#FireFoxProfile.set_preference("general.useragent.override", user_agent)
url = "https://www.ebay.com/"
browser = wd
browser.implicitly_wait(7)
browser.get(url)

search_keyword = 'Playstation 5'
search_field = browser.find_element_by_id('gh-ac')
search_field.clear()
search_field.send_keys(search_keyword)
search_field.send_keys(Keys.ENTER)

total_result = int(browser.find_element_by_class_name('srp-controls__count-heading').find_element_by_class_name('BOLD').text.replace(',', ''))
ebay_listings = []

current_page = 1
to_continue = True

In [7]:
while to_continue:
    try:
        print('Processing page {0}'.format(current_page))

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        item_list = soup.find_all('li', 's-item--watch-at-corner')
        
        for listing in item_list:
            product_detail = {}

            product_detail['product title'] = listing.h3.text
            product_detail['product url'] = listing.a['href']
            
            listing_subtitles = listing.find_all('div', 's-item__subtitle')
            if len(listing_subtitles) == 2:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = listing_subtitles[1].text
            elif len(listing_subtitles) == 1:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = None
            else:
                listing_subtitle_1 = None
                listing_subtitle_2 = None

            product_detail['subtitle1'] = listing_subtitle_1
            product_detail['subtitle2'] = listing_subtitle_2


            # Star rating and product review count
            try:
                x_star_rating = listing.find('div', 'x-star-rating')
                stars = len(x_star_rating.find_all('svg', 'icon--star-filled-small'))
                review_count = int(listing.find('span', 's-item__reviews-count').span.text.replace( ' product ratings', ''))
                
                product_detail['stars'] = stars
                product_detail['product review_count'] = review_count
            except Exception as e:
                product_detail['stars'] = ''
                product_detail['product review_count'] = ''

            product_detail['price'] = listing.find('span', 's-item__price').text

            try:
                product_detail['shipping detail'] = listing.find('span', 's-item__logisticsCost').text
            except AttributeError:
                product_detail['shipping detail'] = 'Not Available'
                
            ebay_listings.append(product_detail)


        time.sleep(1)
        browser.find_element_by_xpath('//a[@class="pagination__item" and text()="{0}"]'.format(current_page+1)).click()
        current_page +=1

        if current_page > 5:
            raise Exception('Stop')

    except NoSuchElementException:
        print('Last page {0}'.format(current_page))
        to_continue = False



Processing page 1
Processing page 2
Processing page 3
Processing page 4
Processing page 5
Last page 5


In [31]:
df = pd.DataFrame(ebay_listings)
df.head()

In [28]:
#cleaner_price = lambda x: int("".join(filter(str.isdigit, x)))
#df.amount = 0
df['amount'] = df.price.apply(lambda x: int("".join(filter(str.isdigit, x))))

In [31]:
df.sort_values(by='amount')

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail,amount
263,Sony Playstation 5 Blu-ray Edition 825gb Conso...,https://www.ebay.com/itm/133894832474?epid=140...,New (Other),None,,,NT$ 617,Shipping not specified,617
182,New ListingPlayStation 5 Disc Edition Console ...,https://www.ebay.com/itm/304174282576?epid=240...,Brand New · Sony · Sony PlayStation 5 · Sony P...,None,0,243,"NT$ 15,295","+NT$ 3,208 shipping estimate",15295
19,New ListingSony PlayStation 5 Disc Edition Cus...,https://www.ebay.com/itm/174972254900?epid=240...,Pre-Owned · Sony · Sony PlayStation 5 · Sony P...,None,0,243,"NT$ 15,677","+NT$ 2,517 shipping estimate",15677
79,New ListingSony PlayStation 5 Disc Edition Cus...,https://www.ebay.com/itm/174972254900?epid=240...,Pre-Owned · Sony · Sony PlayStation 5 · Sony P...,None,0,243,"NT$ 15,677","+NT$ 2,517 shipping estimate",15677
115,"Sony PS5 Digital Edition Console - NEW IN BOX,...",https://www.ebay.com/itm/284480903089?epid=250...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,458,"NT$ 16,543","+NT$ 2,152 shipping estimate",16543
...,...,...,...,...,...,...,...,...,...
191,Sony Playstation 5 + deathloop + Death strandi...,https://www.ebay.com/itm/234227290719?epid=504...,Brand New · Sony,None,0,5,"NT$ 29,206",Shipping not specified,29206
172,SONY Playstation 5 SONY PS5 Disc BRAND NEW *UP...,https://www.ebay.com/itm/393534744747?epid=504...,MAKE SURE ENTER CODE SPENDNSAVEOCT WHEN YOU CH...,Brand New · Sony · Sony PlayStation 5 · Sony P...,0,64,"NT$ 29,520",Shipping not specified,29520
221,Playstation 5 Console Disc Edition Bundle,https://www.ebay.com/itm/114995275163?hash=ite...,Brand New · Sony · Sony PlayStation 5 · Sony P...,None,,,"NT$ 30,552",Shipping not specified,30552
151,Sony PS5 PlayStation 5 Disc Ratchet & Clank: R...,https://www.ebay.com/itm/165108360287?hash=ite...,Brand New · Sony · Sony PlayStation 5 · Sony P...,None,,,"NT$ 34,414","+NT$ 3,821 shipping estimate",34414


In [ ]:
df.to_excel('playstation 5 listings.xlsx', index=False)